### Exploring The Impact of Optimizers and Activation Functions On OODN 

In [3]:
from os import listdir

import time
import json

import torch

from torchvision.datasets import mnist, FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam
from torch.nn import Module
from torch import nn
from torch.nn import CrossEntropyLoss
from torchvision.models.resnet import Bottleneck, ResNet

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from openood.evaluators import metrics

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### Supported Activation Functions

For activation functions, we are considering ReLU, Softplus, Swish. *Note that we may conduct experiments for a subset based on the compute resources available*

In [5]:
def get_activation_fn(activation):
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'softplus':
        return nn.Softplus()
    elif activation == 'swish':
        return nn.Swish()
    return None

### Supported Networks

Currently, we support LeNet and ResNet50.

### LeNet

In [6]:
class LeNet(nn.Module):
    def __init__(self, num_classes, num_channel=3, activation='relu'):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.feature_size = 84
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channel,
                      out_channels=6,
                      kernel_size=5,
                      stride=1,
                      padding=2), get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
             get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=120,
                      kernel_size=5,
                      stride=1), get_activation_fn(activation))

        self.classifier1 = nn.Linear(in_features=120, out_features=84)
        self.relu = get_activation_fn(activation)
        self.fc = nn.Linear(in_features=84, out_features=num_classes)

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()

    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        logits_cls = self.fc(feature)
        feature_list = [feature1, feature2, feature3, feature]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        feature = feature.clip(max=threshold)
        logits_cls = self.fc(feature)

        return logits_cls

### ResNet50

In [7]:
class ResNet50(ResNet):
    def __init__(self,
                 block=Bottleneck,
                 layers=[3, 4, 6, 3],
                 num_classes=1000):
        super(ResNet50, self).__init__(block=block,
                                       layers=layers,
                                       num_classes=num_classes)
        self.feature_size = 2048


    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.view(feature5.size(0), -1)
        logits_cls = self.fc(feature)

        feature_list = [feature1, feature2, feature3, feature4, feature5]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.clip(max=threshold)
        feature = feature.view(feature.size(0), -1)
        logits_cls = self.fc(feature)

        return logits_cls

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()
    
def get_resnet_model(activation_function_type, n_classes):
    resnet_model = ResNet50(num_classes=n_classes)
    resnet_model.to(device)    
    return resnet_model

def set_activation_function(resnet_model, activation_function_type):
    resnet_model.relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[2].relu = get_activation_fn(activation_function_type)

    resnet_model.layer2[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[3].relu = get_activation_fn(activation_function_type)

    resnet_model.layer3[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[3].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[4].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[5].relu = get_activation_fn(activation_function_type)


    resnet_model.layer4[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[2].relu = get_activation_fn(activation_function_type)
    
    return resnet_model

In [8]:
def get_model(config):
    activation_function_type = config["activation_function_type"]
    network_type = config["network"]
    n_classes = config["n_classes"]
    
    if network_type == "lenet":
        model =  LeNet(num_classes=n_classes, num_channel=1, activation=activation_function_type)
    elif network_type == "resnet50":
        model = get_resnet_model(activation_function_type, n_classes)
    else:
        raise Exception("Currently we only support lenet or resnet50")
    
    return model
    

### Supported Post-Hoc OODN Processors

#### The first post processor we consider is ODIN

In [9]:
class OODPostprocessor():
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

In [10]:
class ODINPostprocessor(OODPostprocessor):
    def __init__(self, temperature, noise):
        super(OODPostprocessor)
        self.temperature = temperature
        self.noise = noise
        
    def postprocess(self, net: nn.Module, data):
        net.eval()
        data.requires_grad = True
        output = net(data)

        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        criterion = nn.CrossEntropyLoss()

        labels = output.detach().argmax(axis=1)

        # Using temperature scaling
        output = output / self.temperature

        loss = criterion(output, labels)
        loss.backward()

        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(data.grad.detach(), 0)
        gradient = (gradient.float() - 0.5) * 2

        # Scaling values taken from original code       
        gradient[:, 0] = (gradient[:, 0]) / (63.0 / 255.0)
        if gradient.shape[1] == 3:
            gradient[:, 1] = (gradient[:, 1]) / (62.1 / 255.0)
            gradient[:, 2] = (gradient[:, 2]) / (66.7 / 255.0)

        # Adding small perturbations to images
        tempInputs = torch.add(data.detach(), gradient, alpha=-self.noise)
        output = net(tempInputs)
        output = output / self.temperature

        # Calculating the confidence after adding perturbations
        nnOutput = output.detach()
        nnOutput = nnOutput - nnOutput.max(dim=1, keepdims=True).values
        nnOutput = nnOutput.exp() / nnOutput.exp().sum(dim=1, keepdims=True)

        conf, pred = nnOutput.max(dim=1)

        return pred, conf

#### We consider the Maximum Classifier Discrepancy Post OODN method

https://arxiv.org/pdf/1712.02560.pdf

In [11]:
class MCDPostprocessor(OODPostprocessor):
    def __init__(self, samples: int = 30):
        super(OODPostprocessor)
        self.samples = samples  #: number :math:`N` of samples

    def postprocess(self, model: torch.nn.Module, x: torch.Tensor) -> torch.Tensor:
        mode_switch = False
        if not model.training:
            mode_switch = True

            model.train()

            for mod in model.modules():
                # reset batch norm layers.
                # TODO: are there other layers?
                if isinstance(mod, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
                    mod.train(False)

        results = None
        with torch.no_grad():
            for i in range(self.samples):
                output = model(x).softmax(dim=1)
                if results is None:
                    results = torch.zeros(size=output.shape).to(device)
                results += output
        results /= self.samples

        if mode_switch:
            model.eval()
        
        conf, pred = results.max(dim=1)

        return pred, conf

In [12]:
def get_postprocessor(postprocessor_type="odin"):
    if postprocessor_type == "odin":
        postprocessor = ODINPostprocessor(1000, 0.0014)
    elif postprocessor_type == "mcd":
        postprocessor = MCDPostprocessor(30)
    return postprocessor

### Supported Out of Distribution Detection Metrics

What metrics do we specifically care about here?

**FPR@95** measures the false positive rate (FPR) when the true positive rate (TPR) is
equal to 95%. Lower scores indicate better performance. 

**AUROC** measures the area under the
Receiver Operating Characteristic (ROC) curve, which displays the relationship between TPR and
FPR. The area under the ROC curve can be interpreted as the probability that a positive ID example
will have a higher detection score than a negative OOD example. 

**AUPR** measures the area under
the Precision-Recall (PR) curve. The PR curve is created by plotting precision versus recall. Similar
to AUROC, we consider ID samples as positive, so that the score corresponds to the AUPR-In metric
in some works

In [13]:
def calculate_oodn_metrics(model, postprocessor_type, id_test_loader, ood_test_loader, ood_name):
    postprocessor = get_postprocessor(postprocessor_type)
    id_pred, id_conf, id_gt = postprocessor.inference(
                model, id_test_loader)

    ood_pred, ood_conf, ood_gt = postprocessor.inference(
        model, ood_test_loader)

    ood_gt = -1 * np.ones_like(ood_gt)  # hard set to -1 as ood
    pred = np.concatenate([id_pred, ood_pred])
    conf = np.concatenate([id_conf, ood_conf])
    label = np.concatenate([id_gt, ood_gt])
    ood_metrics = metrics.compute_all_metrics(conf, label, pred)

    return print_and_get_formatted_metrics(ood_metrics, ood_name)

def print_and_get_formatted_metrics(metrics, dataset_name):
    [fpr, auroc, aupr_in, aupr_out,
     ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
     = metrics

    write_content = {
        'dataset': dataset_name,
        'FPR@95': '{:.2f}'.format(100 * fpr),
        'AUROC': '{:.2f}'.format(100 * auroc),
        'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
        'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
        'CCR_4': '{:.2f}'.format(100 * ccr_4),
        'CCR_3': '{:.2f}'.format(100 * ccr_3),
        'CCR_2': '{:.2f}'.format(100 * ccr_2),
        'CCR_1': '{:.2f}'.format(100 * ccr_1),
        'ACC': '{:.2f}'.format(100 * accuracy)
    }

    fieldnames = list(write_content.keys())

    # print ood metric results
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
          end=' ',
          flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
          flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
          end=' ',
          flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)
    return write_content

def load_results_into_df(dir_path):
    res_files = [dir_path+each for each in listdir(dir_path)]
    all_results = []
    columns = ['optimizer_type', 'activation_function_type', 'postprocessor_type', 'trial', 'AUROC', 'ACC']
    for fp in res_files:
        f = open(fp)
        data = json.load(f)
        for trial, results in data.items():
            all_results.append([
                    results['optimizer_type'],
                    results['activation_function_type'],
                    results['postprocessor_type'],
                    trial,
                    float(results['AUROC']),
                    float(results['ACC'])
                ])
    df = pd.DataFrame(all_results, columns=columns)
    return df

In [14]:
def get_optimizer(model, config):
    params = model.parameters()
    lr = config['lr']
    momentum = config['momentum']
    weight_decay = config['weight_decay']
    optimizer_type = config['optimizer_type']
    
    print(f'Getting optimizer for type: {optimizer_type}...')
    if optimizer_type == 'SGD':
        return SGD(params, 
              lr=lr, 
              momentum=momentum,
              weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        return Adam(params, 
                    lr=lr, 
                    weight_decay=weight_decay)
    else:
        raise Exception("Invalid optimizer_type provided, only SGD and Adam are supported currently")

def get_wilds_loader(dataset, split, batch_size):
    d = dataset.get_subset(
        split,
        frac=0.1,
        transform=transforms.Compose(
            [transforms.Resize((448, 448)), transforms.ToTensor()]
        ),
    )
    # Prepare the standard data loader
    return get_train_loader("standard", d, batch_size=batch_size, num_workers=4, pin_memory=True)

def get_data_loaders(config):
    data_loaders = {}
    dataset_name = config["dataset_name"]
    dataset_type = config["dataset_type"]
    batch_size = config['batch_size']
    
    wilds_id_test_split = "id_val" if dataset_name == "camelyon17" else "id_test"
    if dataset_type == "wilds":
        # wilds dataset
        dataset = get_dataset(dataset=dataset_name, download=True)
        
        data_loaders["train"] = get_wilds_loader(dataset, "train", batch_size)
        data_loaders["ood_test"] = get_wilds_loader(dataset, "test", batch_size)
        data_loaders["id_test"] = get_wilds_loader(dataset, wilds_id_test_split, batch_size)
    elif dataset_name == "mnist":
        # mnist dataset
        train_dataset = mnist.MNIST(root='data', download=True, train=True, transform=ToTensor())
        test_dataset = mnist.MNIST(root='data', download=True, train=False, transform=ToTensor())
        fashion_test_dataset = mnist.FashionMNIST(root='data', download=True,train=False,transform=ToTensor())

        data_loaders["train"] = DataLoader(train_dataset, batch_size=batch_size)
        data_loaders["id_test"] = DataLoader(test_dataset, batch_size=batch_size)
        data_loaders["ood_test"] = DataLoader(fashion_test_dataset, batch_size=batch_size)
    
    return data_loaders

In [15]:
def train_resnet_model_given_opti_activation_fn(config):
    # get the train loader
    train_loader = config["data_loaders"]["train"]
    
    # get the resnet model with the replaced activation functions
    model = get_model(config)
    model.to(device)
    
    # get the optimizer
    sgd = get_optimizer(model, config)
    
    loss_fn = CrossEntropyLoss()

    for current_epoch in range(config['epochs']):
        model.train()
        epoch_start = time.time()
        print('Training epoch: {}'.format(current_epoch))
        
        time_per_hundred = time.time()
        for idx, (loader_data) in enumerate(train_loader):
            train_x, train_label = loader_data[0].cuda(), loader_data[1].cuda()
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            if idx % 100 == 0:
                print('idx: {}, loss: {}, time taken: {}'.format(idx, loss.sum().item(), time.time()-time_per_hundred))
                time_per_hundred = time.time()
            loss.backward()
            sgd.step()
        print(f"Time take for epoch {current_epoch}: {time.time() - epoch_start}s")
    
    torch.save(model, config['model_name'])
    return model

def run_full_oodn_pipeline(config):
    metrics = {}
    for i in range(config["trials"]):
        t = time.time()
        model_name = f"models/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}_{i}.pkl"
        print(f'Running model: {model_name}...')
        config['model_name'] = model_name
        # train model
        model = train_resnet_model_given_opti_activation_fn(config)
        # calculate oodn metrics
        metrics[i] = calculate_oodn_metrics(model,
                               config['postprocessor_type'],
                               config["data_loaders"]["id_test"], 
                               config["data_loaders"]["ood_test"], 
                               config["dataset_name"])
        metrics[i]['optimizer_type'] = config['optimizer_type']
        metrics[i]['activation_function_type'] = config['activation_function_type']
        metrics[i]['postprocessor_type'] = config['postprocessor_type']
        metrics[i]['time_taken'] = time.time() - t
        print(f"Time taken to train: {metrics[i]['time_taken']}")
        
    experiment_name = f"{config['results_dir']}/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}.json"
    with open(experiment_name, 'w') as fp:
        json.dump(metrics, fp)
    return metrics

### Study 1: LeNet5, MNIST as the ID Dataset, FashionMNIST as the OOD Dataset

#### Study 1(a): Combination: SGD + ReLU

#### ODIN

In [25]:
config_1a = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "mnist",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1a["data_loaders"] = get_data_loaders(config_1a)
run_full_oodn_pipeline(config_1a)

Running model: models/mnist_lenet_odin_relu_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3107709884643555, time taken: 0.016290903091430664
idx: 100, loss: 0.2591329514980316, time taken: 0.9920833110809326
idx: 200, loss: 0.18179017305374146, time taken: 0.983135461807251
idx: 300, loss: 0.18219703435897827, time taken: 1.2073335647583008
idx: 400, loss: 0.1705045849084854, time taken: 0.9965109825134277
Time take for epoch 0: 4.872047662734985s
Training epoch: 1
idx: 0, loss: 0.049608729779720306, time taken: 0.00919032096862793
idx: 100, loss: 0.0975547730922699, time taken: 1.003889799118042
idx: 200, loss: 0.06459632515907288, time taken: 0.9914870262145996
idx: 300, loss: 0.07444804161787033, time taken: 0.9995100498199463
idx: 400, loss: 0.19108620285987854, time taken: 1.000260829925537
Time take for epoch 1: 4.699322462081909s
Training epoch: 2
idx: 0, loss: 0.04568091034889221, time taken: 0.00902700424194336
idx: 100, loss: 0.100879088044

idx: 100, loss: 0.03807500749826431, time taken: 0.9945220947265625
idx: 200, loss: 0.09613689035177231, time taken: 0.9967730045318604
idx: 300, loss: 0.04140741750597954, time taken: 0.9968740940093994
idx: 400, loss: 0.04109680652618408, time taken: 0.9932870864868164
Time take for epoch 9: 4.665702819824219s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 1.03, AUROC: 99.62 AUPR_IN: 99.66, AUPR_OUT: 99.58
CCR: 49.92, 86.67, 94.39, 97.63, ACC: 98.19
──────────────────────────────────────────────────────────────────────
Time taken to train: 49.472259759902954
Running model: models/mnist_lenet_odin_relu_SGD_2.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3102641105651855, time taken: 0.008780956268310547
idx: 100, loss: 0.4469568133354187, time taken: 0.9988772869110107
idx: 200, loss: 0.19983994960784912, time taken: 0.9965133666992188
idx: 300, loss: 0.17732372

{0: {'dataset': 'mnist',
  'FPR@95': '1.50',
  'AUROC': '99.40',
  'AUPR_IN': '99.50',
  'AUPR_OUT': '99.28',
  'CCR_4': '66.49',
  'CCR_3': '83.34',
  'CCR_2': '93.16',
  'CCR_1': '97.26',
  'ACC': '98.21',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_taken': 51.51332950592041},
 1: {'dataset': 'mnist',
  'FPR@95': '1.03',
  'AUROC': '99.62',
  'AUPR_IN': '99.66',
  'AUPR_OUT': '99.58',
  'CCR_4': '49.92',
  'CCR_3': '86.67',
  'CCR_2': '94.39',
  'CCR_1': '97.63',
  'ACC': '98.19',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_taken': 49.472259759902954},
 2: {'dataset': 'mnist',
  'FPR@95': '1.75',
  'AUROC': '99.52',
  'AUPR_IN': '99.57',
  'AUPR_OUT': '99.50',
  'CCR_4': '75.61',
  'CCR_3': '84.23',
  'CCR_2': '92.80',
  'CCR_1': '97.72',
  'ACC': '98.65',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_take

#### MCD

In [26]:
config_1_1a = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "mnist",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1a["data_loaders"] = get_data_loaders(config_1_1a)
run_full_oodn_pipeline(config_1_1a)

Running model: models/mnist_lenet_mcd_relu_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3041369915008545, time taken: 0.00941777229309082
idx: 100, loss: 0.29034051299095154, time taken: 1.01137375831604
idx: 200, loss: 0.20892561972141266, time taken: 0.9974749088287354
idx: 300, loss: 0.14768998324871063, time taken: 1.005979061126709
idx: 400, loss: 0.3825826942920685, time taken: 1.0021300315856934
Time take for epoch 0: 4.716005086898804s
Training epoch: 1
idx: 0, loss: 0.07344602048397064, time taken: 0.00898289680480957
idx: 100, loss: 0.10576103627681732, time taken: 0.9942357540130615
idx: 200, loss: 0.18464992940425873, time taken: 1.0064194202423096
idx: 300, loss: 0.0687417984008789, time taken: 1.0503413677215576
idx: 400, loss: 0.24697965383529663, time taken: 1.0413062572479248
Time take for epoch 1: 4.8107500076293945s
Training epoch: 2
idx: 0, loss: 0.08865907043218613, time taken: 0.009688138961791992
idx: 100, loss: 0.070626161992

idx: 200, loss: 0.14070814847946167, time taken: 0.9845829010009766
idx: 300, loss: 0.10133273154497147, time taken: 0.9909062385559082
idx: 400, loss: 0.18348412215709686, time taken: 0.9801931381225586
Time take for epoch 9: 4.841415166854858s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 23.99, AUROC: 95.97 AUPR_IN: 96.42, AUPR_OUT: 95.47
CCR: 13.00, 34.59, 61.43, 89.24, ACC: 96.47
──────────────────────────────────────────────────────────────────────
Time taken to train: 50.79097890853882
Running model: models/mnist_lenet_mcd_relu_SGD_2.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3208749294281006, time taken: 0.009003639221191406
idx: 100, loss: 0.6104536652565002, time taken: 1.0265252590179443
idx: 200, loss: 0.17620675265789032, time taken: 1.0480754375457764
idx: 300, loss: 0.12093239277601242, time taken: 0.9974884986877441
idx: 400, loss: 0.245636060

{0: {'dataset': 'mnist',
  'FPR@95': '5.48',
  'AUROC': '98.82',
  'AUPR_IN': '98.97',
  'AUPR_OUT': '98.73',
  'CCR_4': '26.89',
  'CCR_3': '72.35',
  'CCR_2': '87.60',
  'CCR_1': '94.91',
  'ACC': '96.99',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_taken': 50.958593130111694},
 1: {'dataset': 'mnist',
  'FPR@95': '23.99',
  'AUROC': '95.97',
  'AUPR_IN': '96.42',
  'AUPR_OUT': '95.47',
  'CCR_4': '13.00',
  'CCR_3': '34.59',
  'CCR_2': '61.43',
  'CCR_1': '89.24',
  'ACC': '96.47',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_taken': 50.79097890853882},
 2: {'dataset': 'mnist',
  'FPR@95': '8.94',
  'AUROC': '98.45',
  'AUPR_IN': '98.68',
  'AUPR_OUT': '98.26',
  'CCR_4': '61.72',
  'CCR_3': '72.82',
  'CCR_2': '86.09',
  'CCR_1': '94.66',
  'ACC': '97.95',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_taken'

#### 1 (b) SGD + SoftPlus

In [27]:
config_1b = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1b["data_loaders"] = get_data_loaders(config_1b)
run_full_oodn_pipeline(config_1b)

Running model: models/mnist_lenet_odin_softplus_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3718485832214355, time taken: 0.009566783905029297
idx: 100, loss: 2.3032381534576416, time taken: 1.0866444110870361
idx: 200, loss: 1.3475463390350342, time taken: 1.063671588897705
idx: 300, loss: 0.2671454846858978, time taken: 1.0491371154785156
idx: 400, loss: 0.47210097312927246, time taken: 1.2400872707366943
Time take for epoch 0: 5.3507068157196045s
Training epoch: 1
idx: 0, loss: 0.14156891405582428, time taken: 0.012911796569824219
idx: 100, loss: 0.11166047304868698, time taken: 1.2283711433410645
idx: 200, loss: 0.33546292781829834, time taken: 1.047506332397461
idx: 300, loss: 0.06544164568185806, time taken: 1.0868539810180664
idx: 400, loss: 0.24700260162353516, time taken: 1.0451445579528809
Time take for epoch 1: 5.132681846618652s
Training epoch: 2
idx: 0, loss: 0.07973635196685791, time taken: 0.009329080581665039
idx: 100, loss: 0.04751

idx: 200, loss: 0.06157349422574043, time taken: 1.336702585220337
idx: 300, loss: 0.03858104720711708, time taken: 1.3417222499847412
idx: 400, loss: 0.13049939274787903, time taken: 1.291152000427246
Time take for epoch 9: 5.840482950210571s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 4.65, AUROC: 98.65 AUPR_IN: 98.95, AUPR_OUT: 98.14
CCR: 36.44, 71.84, 88.73, 95.84, ACC: 97.92
──────────────────────────────────────────────────────────────────────
Time taken to train: 55.54051423072815
Running model: models/mnist_lenet_odin_softplus_SGD_2.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3162033557891846, time taken: 0.015664100646972656
idx: 100, loss: 2.3032705783843994, time taken: 1.138742208480835
idx: 200, loss: 0.510703980922699, time taken: 1.0561707019805908
idx: 300, loss: 0.4456235468387604, time taken: 1.0522348880767822
idx: 400, loss: 0.58482569456

{0: {'dataset': 'mnist',
  'FPR@95': '3.65',
  'AUROC': '99.17',
  'AUPR_IN': '99.19',
  'AUPR_OUT': '99.17',
  'CCR_4': '37.85',
  'CCR_3': '54.76',
  'CCR_2': '86.31',
  'CCR_1': '97.47',
  'ACC': '98.33',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin',
  'time_taken': 53.89000749588013},
 1: {'dataset': 'mnist',
  'FPR@95': '4.65',
  'AUROC': '98.65',
  'AUPR_IN': '98.95',
  'AUPR_OUT': '98.14',
  'CCR_4': '36.44',
  'CCR_3': '71.84',
  'CCR_2': '88.73',
  'CCR_1': '95.84',
  'ACC': '97.92',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin',
  'time_taken': 55.54051423072815},
 2: {'dataset': 'mnist',
  'FPR@95': '3.61',
  'AUROC': '99.23',
  'AUPR_IN': '99.30',
  'AUPR_OUT': '99.19',
  'CCR_4': '35.25',
  'CCR_3': '75.29',
  'CCR_2': '89.49',
  'CCR_1': '97.23',
  'ACC': '98.51',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin',
 

In [28]:
config_1_1b = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1b["data_loaders"] = get_data_loaders(config_1_1b)
run_full_oodn_pipeline(config_1_1b)

Running model: models/mnist_lenet_mcd_softplus_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3969497680664062, time taken: 0.02117300033569336
idx: 100, loss: 2.305872678756714, time taken: 1.0806331634521484
idx: 200, loss: 2.323749542236328, time taken: 1.059751033782959
idx: 300, loss: 2.2967991828918457, time taken: 1.0730648040771484
idx: 400, loss: 1.9109351634979248, time taken: 1.0473692417144775
Time take for epoch 0: 5.007572650909424s
Training epoch: 1
idx: 0, loss: 1.2711639404296875, time taken: 0.011802911758422852
idx: 100, loss: 0.3356267809867859, time taken: 1.0962979793548584
idx: 200, loss: 0.39382028579711914, time taken: 1.0523560047149658
idx: 300, loss: 0.37495359778404236, time taken: 1.0539612770080566
idx: 400, loss: 0.4912671148777008, time taken: 1.0574727058410645
Time take for epoch 1: 4.9890546798706055s
Training epoch: 2
idx: 0, loss: 0.23660379648208618, time taken: 0.011170148849487305
idx: 100, loss: 0.307679057121

idx: 200, loss: 0.07735861092805862, time taken: 1.0630450248718262
idx: 300, loss: 0.06182503327727318, time taken: 1.0735399723052979
idx: 400, loss: 0.11808710545301437, time taken: 1.064002513885498
Time take for epoch 9: 4.9802563190460205s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 7.12, AUROC: 98.30 AUPR_IN: 98.70, AUPR_OUT: 97.81
CCR: 48.13, 77.56, 90.23, 95.28, ACC: 98.12
──────────────────────────────────────────────────────────────────────
Time taken to train: 55.63301873207092
Running model: models/mnist_lenet_mcd_softplus_SGD_2.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3448333740234375, time taken: 0.011051654815673828
idx: 100, loss: 2.305781841278076, time taken: 1.0472984313964844
idx: 200, loss: 2.319713592529297, time taken: 1.0453503131866455
idx: 300, loss: 1.990025281906128, time taken: 1.083644151687622
idx: 400, loss: 1.080050110816

{0: {'dataset': 'mnist',
  'FPR@95': '9.90',
  'AUROC': '98.29',
  'AUPR_IN': '98.55',
  'AUPR_OUT': '98.08',
  'CCR_4': '43.40',
  'CCR_3': '70.02',
  'CCR_2': '85.26',
  'CCR_1': '94.45',
  'ACC': '97.86',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd',
  'time_taken': 56.9134042263031},
 1: {'dataset': 'mnist',
  'FPR@95': '7.12',
  'AUROC': '98.30',
  'AUPR_IN': '98.70',
  'AUPR_OUT': '97.81',
  'CCR_4': '48.13',
  'CCR_3': '77.56',
  'CCR_2': '90.23',
  'CCR_1': '95.28',
  'ACC': '98.12',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd',
  'time_taken': 55.63301873207092},
 2: {'dataset': 'mnist',
  'FPR@95': '25.25',
  'AUROC': '96.59',
  'AUPR_IN': '97.55',
  'AUPR_OUT': '95.23',
  'CCR_4': '39.79',
  'CCR_3': '75.68',
  'CCR_2': '85.35',
  'CCR_1': '92.15',
  'ACC': '98.27',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd',
  't

#### 1(c) Adam + ReLU

In [29]:
config_1c = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1c["data_loaders"] = get_data_loaders(config_1c)
run_full_oodn_pipeline(config_1c)

Running model: models/mnist_lenet_odin_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.302553653717041, time taken: 0.013901472091674805
idx: 100, loss: 0.28000783920288086, time taken: 1.118457555770874
idx: 200, loss: 0.1414969563484192, time taken: 1.1110692024230957
idx: 300, loss: 0.12729781866073608, time taken: 1.127894401550293
idx: 400, loss: 0.29341307282447815, time taken: 1.1033143997192383
Time take for epoch 0: 5.232776880264282s
Training epoch: 1
idx: 0, loss: 0.1049453392624855, time taken: 0.011766672134399414
idx: 100, loss: 0.05889567360281944, time taken: 1.1290693283081055
idx: 200, loss: 0.1544564962387085, time taken: 1.1138193607330322
idx: 300, loss: 0.09296706318855286, time taken: 1.092811107635498
idx: 400, loss: 0.20761510729789734, time taken: 1.0934405326843262
Time take for epoch 1: 5.198179483413696s
Training epoch: 2
idx: 0, loss: 0.04594803601503372, time taken: 0.013196229934692383
idx: 100, loss: 0.0420266762

idx: 100, loss: 0.02613396756350994, time taken: 1.216752052307129
idx: 200, loss: 0.100645512342453, time taken: 1.1088371276855469
idx: 300, loss: 0.07560759037733078, time taken: 1.1201374530792236
idx: 400, loss: 0.09658131748437881, time taken: 1.1147878170013428
Time take for epoch 9: 5.325292110443115s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 2.80, AUROC: 99.26 AUPR_IN: 99.32, AUPR_OUT: 99.23
CCR: 43.84, 66.05, 89.83, 96.78, ACC: 97.88
──────────────────────────────────────────────────────────────────────
Time taken to train: 56.29766273498535
Running model: models/mnist_lenet_odin_relu_Adam_2.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.313971996307373, time taken: 0.012202262878417969
idx: 100, loss: 0.14131981134414673, time taken: 1.1448302268981934
idx: 200, loss: 0.1397102177143097, time taken: 1.1313438415527344
idx: 300, loss: 0.11277240514

{0: {'dataset': 'mnist',
  'FPR@95': '1.75',
  'AUROC': '99.29',
  'AUPR_IN': '99.40',
  'AUPR_OUT': '99.17',
  'CCR_4': '55.04',
  'CCR_3': '84.44',
  'CCR_2': '92.46',
  'CCR_1': '96.19',
  'ACC': '97.44',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_taken': 58.74810719490051},
 1: {'dataset': 'mnist',
  'FPR@95': '2.80',
  'AUROC': '99.26',
  'AUPR_IN': '99.32',
  'AUPR_OUT': '99.23',
  'CCR_4': '43.84',
  'CCR_3': '66.05',
  'CCR_2': '89.83',
  'CCR_1': '96.78',
  'ACC': '97.88',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_taken': 56.29766273498535},
 2: {'dataset': 'mnist',
  'FPR@95': '0.07',
  'AUROC': '99.86',
  'AUPR_IN': '99.88',
  'AUPR_OUT': '99.85',
  'CCR_4': '89.81',
  'CCR_3': '94.73',
  'CCR_2': '97.14',
  'CCR_1': '98.20',
  'ACC': '98.39',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin',
  'time_ta

In [30]:
config_1_1c = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1c["data_loaders"] = get_data_loaders(config_1_1c)
run_full_oodn_pipeline(config_1_1c)

Running model: models/mnist_lenet_mcd_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.315467596054077, time taken: 0.01088714599609375
idx: 100, loss: 0.18580283224582672, time taken: 1.1522655487060547
idx: 200, loss: 0.15429630875587463, time taken: 1.0996689796447754
idx: 300, loss: 0.09404754638671875, time taken: 1.1279420852661133
idx: 400, loss: 0.2952556312084198, time taken: 1.1384093761444092
Time take for epoch 0: 5.325622081756592s
Training epoch: 1
idx: 0, loss: 0.10620678961277008, time taken: 0.00960683822631836
idx: 100, loss: 0.13662971556186676, time taken: 1.154064655303955
idx: 200, loss: 0.1432105302810669, time taken: 1.110403060913086
idx: 300, loss: 0.062432173639535904, time taken: 1.1145029067993164
idx: 400, loss: 0.17716443538665771, time taken: 1.1192762851715088
Time take for epoch 1: 5.3026511669158936s
Training epoch: 2
idx: 0, loss: 0.09528520703315735, time taken: 0.009609460830688477
idx: 100, loss: 0.114968128

idx: 200, loss: 0.0923265889286995, time taken: 1.0961582660675049
idx: 300, loss: 0.059856709092855453, time taken: 1.1508381366729736
idx: 400, loss: 0.07363149523735046, time taken: 1.1946306228637695
Time take for epoch 9: 5.309803247451782s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 4.14, AUROC: 99.09 AUPR_IN: 99.21, AUPR_OUT: 99.00
CCR: 68.42, 80.19, 90.49, 96.49, ACC: 98.20
──────────────────────────────────────────────────────────────────────
Time taken to train: 59.01862144470215
Running model: models/mnist_lenet_mcd_relu_Adam_2.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.3059170246124268, time taken: 0.00960993766784668
idx: 100, loss: 0.1333780288696289, time taken: 1.1127848625183105
idx: 200, loss: 0.14914080500602722, time taken: 1.1316158771514893
idx: 300, loss: 0.08159365504980087, time taken: 1.1259143352508545
idx: 400, loss: 0.213765248

{0: {'dataset': 'mnist',
  'FPR@95': '9.19',
  'AUROC': '98.21',
  'AUPR_IN': '98.39',
  'AUPR_OUT': '98.12',
  'CCR_4': '34.25',
  'CCR_3': '49.21',
  'CCR_2': '80.74',
  'CCR_1': '94.50',
  'ACC': '97.84',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_taken': 59.438806772232056},
 1: {'dataset': 'mnist',
  'FPR@95': '4.14',
  'AUROC': '99.09',
  'AUPR_IN': '99.21',
  'AUPR_OUT': '99.00',
  'CCR_4': '68.42',
  'CCR_3': '80.19',
  'CCR_2': '90.49',
  'CCR_1': '96.49',
  'ACC': '98.20',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_taken': 59.01862144470215},
 2: {'dataset': 'mnist',
  'FPR@95': '12.91',
  'AUROC': '97.89',
  'AUPR_IN': '98.29',
  'AUPR_OUT': '97.56',
  'CCR_4': '54.88',
  'CCR_3': '73.70',
  'CCR_2': '85.49',
  'CCR_1': '93.06',
  'ACC': '96.87',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd',
  'time_tak

#### 1(d) Adam + Softplus

In [45]:
config_1d = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1d["data_loaders"] = get_data_loaders(config_1d)
run_full_oodn_pipeline(config_1d)

Running model: models/mnist_lenet_odin_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.4867539405822754, time taken: 0.01294088363647461
idx: 100, loss: 2.3137519359588623, time taken: 1.0919625759124756
idx: 200, loss: 2.3062570095062256, time taken: 1.1259357929229736
idx: 300, loss: 2.2964324951171875, time taken: 1.0678155422210693
idx: 400, loss: 2.311978816986084, time taken: 1.0967864990234375
Time take for epoch 0: 5.3076536655426025s
Training epoch: 1
idx: 0, loss: 2.2973134517669678, time taken: 0.015149831771850586
idx: 100, loss: 2.3036019802093506, time taken: 1.3365702629089355
idx: 200, loss: 2.3831276893615723, time taken: 1.3034815788269043
idx: 300, loss: 2.297569751739502, time taken: 1.0715382099151611
idx: 400, loss: 2.3055648803710938, time taken: 1.0908071994781494
Time take for epoch 1: 5.661466121673584s
Training epoch: 2
idx: 0, loss: 2.295820951461792, time taken: 0.013700723648071289
idx: 100, loss: 2.303547143936

idx: 200, loss: 0.10514513403177261, time taken: 1.0458641052246094
idx: 300, loss: 0.1100369393825531, time taken: 1.0846061706542969
idx: 400, loss: 0.19702760875225067, time taken: 1.2940168380737305
Time take for epoch 9: 5.201056957244873s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 24.07, AUROC: 95.02 AUPR_IN: 94.94, AUPR_OUT: 95.23
CCR: 9.76, 23.79, 42.58, 84.65, ACC: 96.61
──────────────────────────────────────────────────────────────────────
Time taken to train: 54.05677795410156
Running model: models/mnist_lenet_odin_softplus_Adam_2.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.3333449363708496, time taken: 0.01078653335571289
idx: 100, loss: 1.7437829971313477, time taken: 1.046682357788086
idx: 200, loss: 0.3491598963737488, time taken: 1.0483002662658691
idx: 300, loss: 0.13251887261867523, time taken: 1.0524356365203857
idx: 400, loss: 0.3481087

{0: {'dataset': 'mnist',
  'FPR@95': '39.63',
  'AUROC': '92.68',
  'AUPR_IN': '93.52',
  'AUPR_OUT': '91.72',
  'CCR_4': '4.24',
  'CCR_3': '19.90',
  'CCR_2': '54.15',
  'CCR_1': '77.61',
  'ACC': '93.81',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin',
  'time_taken': 56.324368715286255},
 1: {'dataset': 'mnist',
  'FPR@95': '24.07',
  'AUROC': '95.02',
  'AUPR_IN': '94.94',
  'AUPR_OUT': '95.23',
  'CCR_4': '9.76',
  'CCR_3': '23.79',
  'CCR_2': '42.58',
  'CCR_1': '84.65',
  'ACC': '96.61',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin',
  'time_taken': 54.05677795410156},
 2: {'dataset': 'mnist',
  'FPR@95': '9.58',
  'AUROC': '98.39',
  'AUPR_IN': '98.59',
  'AUPR_OUT': '98.28',
  'CCR_4': '55.18',
  'CCR_3': '69.09',
  'CCR_2': '84.15',
  'CCR_1': '94.21',
  'ACC': '97.71',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin

In [46]:
config_1_1d = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1d["data_loaders"] = get_data_loaders(config_1_1d)
run_full_oodn_pipeline(config_1_1d)

Running model: models/mnist_lenet_mcd_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.353691816329956, time taken: 0.018357038497924805
idx: 100, loss: 2.3137669563293457, time taken: 1.4791052341461182
idx: 200, loss: 1.5023689270019531, time taken: 1.3667633533477783
idx: 300, loss: 0.44497811794281006, time taken: 1.3123645782470703
idx: 400, loss: 0.4015829861164093, time taken: 1.1107428073883057
Time take for epoch 0: 5.999896049499512s
Training epoch: 1
idx: 0, loss: 0.11189687997102737, time taken: 0.010910987854003906
idx: 100, loss: 0.15927621722221375, time taken: 1.0646929740905762
idx: 200, loss: 0.1227123960852623, time taken: 1.0906193256378174
idx: 300, loss: 0.08286695927381516, time taken: 1.060987949371338
idx: 400, loss: 0.21853095293045044, time taken: 1.0454885959625244
Time take for epoch 1: 5.025881052017212s
Training epoch: 2
idx: 0, loss: 0.06880941987037659, time taken: 0.012098073959350586
idx: 100, loss: 0.114383

idx: 100, loss: 0.03830929473042488, time taken: 1.0471446514129639
idx: 200, loss: 0.06321410834789276, time taken: 1.1186833381652832
idx: 300, loss: 0.060200389474630356, time taken: 1.222710132598877
idx: 400, loss: 0.205159991979599, time taken: 1.264312505722046
Time take for epoch 9: 5.498885154724121s
Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 29.09, AUROC: 94.62 AUPR_IN: 95.05, AUPR_OUT: 94.23
CCR: 10.40, 24.61, 52.52, 85.33, ACC: 97.42
──────────────────────────────────────────────────────────────────────
Time taken to train: 57.39354372024536
Running model: models/mnist_lenet_mcd_softplus_Adam_2.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.428304433822632, time taken: 0.01504826545715332
idx: 100, loss: 1.0391908884048462, time taken: 1.1295280456542969
idx: 200, loss: 0.24591122567653656, time taken: 1.0662751197814941
idx: 300, loss: 0.14067403

{0: {'dataset': 'mnist',
  'FPR@95': '41.63',
  'AUROC': '94.48',
  'AUPR_IN': '95.95',
  'AUPR_OUT': '91.68',
  'CCR_4': '34.59',
  'CCR_3': '55.71',
  'CCR_2': '71.79',
  'CCR_1': '88.20',
  'ACC': '97.90',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd',
  'time_taken': 56.13298273086548},
 1: {'dataset': 'mnist',
  'FPR@95': '29.09',
  'AUROC': '94.62',
  'AUPR_IN': '95.05',
  'AUPR_OUT': '94.23',
  'CCR_4': '10.40',
  'CCR_3': '24.61',
  'CCR_2': '52.52',
  'CCR_1': '85.33',
  'ACC': '97.42',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd',
  'time_taken': 57.39354372024536},
 2: {'dataset': 'mnist',
  'FPR@95': '24.29',
  'AUROC': '95.93',
  'AUPR_IN': '96.53',
  'AUPR_OUT': '95.02',
  'CCR_4': '16.29',
  'CCR_3': '32.34',
  'CCR_2': '66.10',
  'CCR_1': '89.93',
  'ACC': '97.85',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'

### Robustness Analysis For Study 1

In [47]:
df_study_1 = load_results_into_df('mnist-study/')
df_study_1

,optimizer_type,activation_function_type,postprocessor_type,trial,AUROC,ACC
0,SGD,softplus,odin,0,99.17,98.33
1,SGD,softplus,odin,1,98.65,97.92
2,SGD,softplus,odin,2,99.23,98.51
3,Adam,softplus,odin,0,92.68,93.81
4,Adam,softplus,odin,1,95.02,96.61
5,Adam,softplus,odin,2,98.39,97.71
6,Adam,relu,mcd,0,98.21,97.84
7,Adam,relu,mcd,1,99.09,98.20
8,Adam,relu,mcd,2,97.89,96.87
9,SGD,relu,mcd,0,98.82,96.99


In [48]:
df_study_1.groupby(['optimizer_type', 'activation_function_type'])['AUROC'].describe()

count       mean       std    min  \
optimizer_type activation_function_type                                      
Adam           relu                        6.0  98.933333  0.738774  97.89   
               softplus                    6.0  95.186667  1.894494  92.68   
SGD            relu                        6.0  98.630000  1.379072  95.97   
               softplus                    6.0  98.371667  0.963087  96.59   

                                             25%     50%      75%    max  
optimizer_type activation_function_type                                   
Adam           relu                      98.4300  99.175  99.2825  99.86  
               softplus                  94.5150  94.820  95.7025  98.39  
SGD            relu                      98.5425  99.110  99.4900  99.62  
               softplus                  98.2925  98.475  99.0400  99.23

In [49]:
df_study_1.groupby(['activation_function_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
activation_function_type,,,,,,,,
relu,12.0,98.781667,1.066607,95.97,98.39,99.175,99.430,99.86
softplus,12.0,96.779167,2.195365,92.68,94.92,97.440,98.455,99.23


In [50]:
df_study_1.groupby(['optimizer_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
optimizer_type,,,,,,,,
Adam,12.0,97.060000,2.389123,92.68,94.9200,98.050,99.1325,99.86
SGD,12.0,98.500833,1.142051,95.97,98.2975,98.735,99.2725,99.62


In [51]:
df_study_1.groupby(['postprocessor_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
postprocessor_type,,,,,,,,
mcd,12.0,97.220000,1.629311,94.48,95.960,98.050,98.3375,99.09
odin,12.0,98.340833,2.191954,92.68,98.585,99.245,99.4300,99.86


In [16]:
def perform_oodn_analysis_on_preloaded_model(model_path, dataset_name, postprocessor_type, id_test, ood_test):
    model = torch.load(model_path)
    return calculate_oodn_metrics(model,
                                   postprocessor_type,
                                   id_test,
                                   ood_test,
                                   dataset_name)

In [54]:
perform_oodn_analysis_on_preloaded_model('/home/rdr2143/cifar_resnet50_odin_relu_SGD_0.pkl', 'odin', )

Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 45.00, AUROC: 92.68 AUPR_IN: 93.77, AUPR_OUT: 91.00
CCR: 1.08, 25.41, 51.53, 82.04, ACC: 95.29
──────────────────────────────────────────────────────────────────────


In [56]:
perform_oodn_analysis_on_preloaded_model('models/lenet_mnist_softplus_Adam_0.pkl', config_1a)

Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 30.70, AUROC: 93.93 AUPR_IN: 94.30, AUPR_OUT: 93.41
CCR: 3.44, 20.72, 50.02, 81.76, ACC: 95.28
──────────────────────────────────────────────────────────────────────
